In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import hashlib

PATH = 'sales_2019-09-01_2020-12-07.csv'

df = pd.read_csv(PATH, delimiter=',')

df.head()

In [ ]:
df[df['total_sales'] <= 0 ].head(10)

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
# adding average order value

df['average_order_value'] = df['total_sales'] / df['orders']

In [ ]:
df1 = df[df.isna().any(axis=1)]
values = {'customer_name': 'none', 'customer_email': 'none'}
df = df.fillna(value=values)

In [ ]:
# remove all customers that have a total_sales <= 0
df = df[df['total_sales'] > 0]

In [ ]:
total_customer = df['orders'].count()
total_orders_quantity =  df['orders'].sum()

In [ ]:
df['orders'].sum()

In [ ]:
# total purchase freq

total_purchase_freq = total_orders_quantity / total_customer
total_purchase_freq

In [ ]:
# adding customer value

df['customer_value'] = (df['average_order_value'] * total_purchase_freq) / 16

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
lifespan6 = 6
lifespan18 = 18
lifespan3yr = 36

lifespan = [6, 18, 36]



In [ ]:
for l in lifespan:
  df['lifetime_value_' + str(l)] = df['customer_value'] * l

In [ ]:
df.head(50)

## Normalize

In [ ]:
df.describe()

In [ ]:
df_normalized = (df - df.mean()) / df.std()
df_normalized.describe()

In [ ]:
from sklearn.cluster import KMeans

kmeans_settings = {
    'algorithm': 'manual',
    'copy_x': True,
    'init': 'k-means++',
    'max_iter': 300,
    'n_clusters': 'manual',
    'n_init': 10,
    'n_jobs': None,
    'precompute_distances': 'auto',
    'random_state': 40,
    'tol': 0.0001,
    'verbose': 0
}

# kmeans = KMeans(**kmeans_settings).fit(df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]])

X = df_normalized[['average_order_value', 'customer_value', 'orders', 'total_sales', 'lifetime_value_6', 'lifetime_value_18', 'lifetime_value_36']]
kmeans = KMeans(n_clusters = 4, random_state=40).fit(X)

In [ ]:
kmeans

In [ ]:
four_cluster_df = X.copy(deep=True)
four_cluster_df['Cluster'] = kmeans.labels_

In [ ]:
four_cluster_df.head()

In [ ]:
four_cluster_df.groupby('Cluster').count().sort_values(by='lifetime_value_6')['lifetime_value_6']

In [ ]:
list(four_cluster_df.columns)

In [ ]:
alt_df = pd.DataFrame()
alt_df['Cluster'] = kmeans.labels_

del four_cluster_df['Cluster']
four_cluster_df

In [ ]:
label

In [ ]:
import itertools
list(itertools.combinations(four_cluster_df.columns, 2))

In [ ]:
col_pairs = list(itertools.combinations(four_cluster_df.columns, 2))
models = []
results = []
for a, b in col_pairs:
  plt.scatter(
    four_cluster_df.loc[alt_df['Cluster'] == 0][str(a)],
    four_cluster_df.loc[alt_df['Cluster'] == 0][str(b)],
    c='blue'
  )
  plt.scatter(
    four_cluster_df.loc[alt_df['Cluster'] == 1][str(a)],
    four_cluster_df.loc[alt_df['Cluster'] == 1][str(b)],
    c='red'
  )
  plt.scatter(
    four_cluster_df.loc[alt_df['Cluster'] == 2][str(a)],
    four_cluster_df.loc[alt_df['Cluster'] == 2][str(b)],
    c='green'
  )
  plt.scatter(
    four_cluster_df.loc[alt_df['Cluster'] == 3][str(a)],
    four_cluster_df.loc[alt_df['Cluster'] == 3][str(b)],
    c='orange'
  )
  plt.title(str(a) + ' vs ' + str(b))
  plt.xlabel(str(b))
  plt.ylabel(str(a))

  plt.grid()
  plt.show()
